In [7]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define the ResNet50 model
resnet50 = torchvision.models.resnet50(pretrained=True)

# Modify the output layer
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Print the modified ResNet50 architecture
print(resnet50)

# Define data transforms



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    #transform.RandomErasing(),
])
# Load datasets
train_dataset = datasets.ImageFolder(root="D:\\Programming\\Dataset_OpenCvDl_Hw2_Q5\\dataset\\training_dataset", transform=transform)
val_dataset = datasets.ImageFolder(root="D:\\Programming\\Dataset_OpenCvDl_Hw2_Q5\\dataset\\validation_dataset", transform=transform)

In [10]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)

# Train the model (example, you may need to customize this part)
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
train_loss_history=[]
val_accuracy_history=[]
for epoch in range(num_epochs):
    resnet50.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

    train_loss_history.append(loss.item())

    # Evaluate the model on the validation set
    resnet50.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet50(inputs)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.float().view_as(predicted)).sum().item()

        accuracy = correct / total
        val_accuracy_history.append(accuracy)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}')

# Save the trained model
torch.save(resnet50.state_dict(), 'resnet50_model.pth')

Epoch [1/10], Loss: 0.1262, Validation Accuracy: 0.7556
Epoch [2/10], Loss: 0.2043, Validation Accuracy: 0.9100
Epoch [3/10], Loss: 0.1255, Validation Accuracy: 0.9433
Epoch [4/10], Loss: 0.0723, Validation Accuracy: 0.9089
Epoch [5/10], Loss: 0.0116, Validation Accuracy: 0.9150
Epoch [6/10], Loss: 0.7971, Validation Accuracy: 0.9094
Epoch [7/10], Loss: 0.0376, Validation Accuracy: 0.9456
Epoch [8/10], Loss: 0.1055, Validation Accuracy: 0.9389
Epoch [9/10], Loss: 0.0200, Validation Accuracy: 0.9494
Epoch [10/10], Loss: 0.1368, Validation Accuracy: 0.9461


In [13]:
print(val_accuracy_history)

[0.7555555555555555, 0.91, 0.9433333333333334, 0.9088888888888889, 0.915, 0.9094444444444445, 0.9455555555555556, 0.9388888888888889, 0.9494444444444444, 0.9461111111111111]


In [14]:
import pandas as pd

df=pd.DataFrame(val_accuracy_history)
df.to_csv("res50_val_acc.csv")